In [1]:
########################################
# K-En Yakın Komşu (K-Nearest Neighbors)
########################################

# Gözlemlerin birbirine olan benzerlikleri üzerinden tahmin yapılır.

In [2]:
# 1. Exploratory Data Analysis
# 2. Data Preprocessing & Feature Engineering
# 3. Modeling & Prediction
# 4. Model Evalution
# 5. Hyperparameter Optimization
# 6. Final Model

In [39]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler,StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_validate,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

pd.set_option("display.max_columns",None)
pd.set_option("display.float_format",lambda x: '%.3f' % x)
pd.set_option("display.width",500)

df = pd.read_csv("diabetes.csv")

In [7]:
######################################################
# 1 - KEŞİFCİ VERİ ANALİZİ (Exploratory Data Analysis)
######################################################
df.head()

df.shape

df.describe().T

df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [13]:
#####################################
# Veri Ön İşleme (Data Preprocessing)
#####################################

y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

X_scaled = StandardScaler().fit_transform(X) 
# X değerlerini standartlaştırdık.

X = pd.DataFrame(X_scaled, columns=X.columns)
# X deki değişkenleri yeni değerler ile değiştirdik ki kolon isimleri gelsin

In [17]:
#############################
# Model & Tahmin (Prediction)
#############################

knn_model = KNeighborsClassifier().fit(X,y)
# Modeli kurduk
# Veri setinde bir öğrenme işlemi gerçekleştrdik.
# Bağımlı, bağımsız değişken arasındaki ilişkiyi öğrenmiş olduk

In [19]:
# Rastgele bir hasta aldık
random_user = X.sample(1,random_state=45)

knn_model.predict(random_user)
# Modele bu kişi diyabet mi diye sorduk.
# array([1]), Hasta dedi

array([1], dtype=int64)

In [23]:
##############################################
# Model Başarı Değerlendirme (Model Evalution)
##############################################

y_pred = knn_model.predict(X) # Bütün gözlem birimleri için tahmin ettik.

y_prob = knn_model.predict_proba(X)[:,1] # 1 sınıfına ait olma olasılıkları

print(classification_report(y,y_pred))
# acc: 0.83
# f1: 0.74

# AUC
roc_auc_score(y,y_prob)
# 0.90

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       500
           1       0.79      0.70      0.74       268

    accuracy                           0.83       768
   macro avg       0.82      0.80      0.81       768
weighted avg       0.83      0.83      0.83       768



0.9017686567164179

In [26]:
# 5 Katlı Çarpaz Doğrulama
cv_result = cross_validate(knn_model, X, y, cv=5, scoring=["accuracy","f1","roc_auc"])
cv_result 

{'fit_time': array([0.00299072, 0.00299144, 0.00198412, 0.00201273, 0.00109267]),
 'score_time': array([0.01451468, 0.0069828 , 0.00560498, 0.00498199, 0.00497794]),
 'test_accuracy': array([0.72077922, 0.73376623, 0.71428571, 0.77124183, 0.7254902 ]),
 'test_f1': array([0.58252427, 0.60952381, 0.54166667, 0.63917526, 0.58      ]),
 'test_roc_auc': array([0.77555556, 0.78759259, 0.73194444, 0.83226415, 0.77528302])}

In [32]:
cv_result["test_accuracy"].mean()
# 0.733112638994992

cv_result["test_f1"].mean()
# 0.5905780011534191

cv_result["test_roc_auc"].mean()
# 0.7805279524807827

0.7805279524807827

In [33]:
# Bu değerler nasıl arttırılabilir.

# 1. Örnek boyutu arttırılabilir.
# 2. Veri ön işleme detaylandırılabilir.
# 3. Özellik mühendisi ile yeni değerler oluşturulabilir.
# 4. İlgili algoritma için optimizasyonlar yapılabilir.

In [35]:
knn_model.get_params()

# Parametre: Modellerin veri içinden öğrendiği ağırlıklardır. Ağırlıklar o parametlerin tahmincileriydi.

# Hiperparametre: Kullanıcı tarafından  tanımlanması gereken dışsal ve veri seti içinden öğrenilmeyen parametrelerdir.

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [38]:
##############################
# Hiperparametre Optimizasyonu
##############################

knn_model.get_params()
# 5 tane komşuluk değeri var
# Komşuluk sayısını değiştirerek, olması gereken en optimum komşuluk sayısını bulmak istiyorum

knn_params = {"n_neighbors": range(2,50)}

In [40]:
knn_gs_best = GridSearchCV(knn_model, knn_params, cv=5, n_jobs=-1, verbose=1).fit(X,y)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [42]:
knn_gs_best.best_params_
# 17 komşuluk sayısı geldi.

{'n_neighbors': 17}

In [44]:
#############
# Final Model
#############
knn_final = knn_model.set_params(**knn_gs_best.best_params_).fit(X,y) # ** dedik direkt atayacak

In [45]:
cv_result = cross_validate(knn_final, X, y, cv=5, scoring=["accuracy","f1","roc_auc"])
cv_result

{'fit_time': array([0.00184774, 0.        , 0.00100684, 0.00110316, 0.00096512]),
 'score_time': array([0.00652122, 0.02352929, 0.00600457, 0.005054  , 0.00497746]),
 'test_accuracy': array([0.75324675, 0.74025974, 0.75974026, 0.81045752, 0.77124183]),
 'test_f1': array([0.60416667, 0.60784314, 0.57471264, 0.70103093, 0.59770115]),
 'test_roc_auc': array([0.80675926, 0.7912037 , 0.79166667, 0.84603774, 0.82830189])}

In [46]:
cv_result["test_accuracy"].mean()
# 0.7669892199303965

cv_result["test_f1"].mean()
# 0.6170909049720137

cv_result["test_roc_auc"].mean()
# 0.8127938504542278

# Değerler artmış

0.7669892199303965